In [14]:
%pylab inline
from collections import Counter
import http.client, urllib.parse, json, time, sys
from glob import glob
from bs4 import BeautifulSoup as Soup
sys.path.append("../words")
import we
from sklearn.svm import LinearSVC, SVC
import numpy as np
import re, sys
import random
import scipy
from adjustText import adjust_text

import statsmodels.stats.proportion
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import sklearn.feature_selection 
from nltk.stem.porter import *
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time
import os, json
import re
import textblob
import langdetect
import pickle as pkl
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from bs4 import BeautifulSoup
from random import randint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
import we
import os

import numpy as np
# from mag.experiment import Experiment
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
# In [2]:
import pandas as pd
import numpy as np

import random
import pickle as pkl
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import scipy


Populating the interactive namespace from numpy and matplotlib


/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
predictions = pkl.load(open('results/joint_predictions_bert.pkl','rb'))
predictions = scipy.special.softmax(predictions)

In [12]:
pr_preds = pkl.load(open('results/balsample__predictions_bert.pkl','rb'))
pr_preds= scipy.special.softmax(pr_preds)

In [128]:
gender_predictions = pkl.load(open('results/gender_predictions_bal_test.pkl','rb'))
gender_predictions = scipy.special.softmax(gender_predictions)

In [50]:
# load bios data
from sklearn.metrics import classification_report,accuracy_score
df_test = pd.read_csv('csvs/multilabel_bios_test.csv')

preds = gender_predictions
preds = np.argmax(preds, axis=1).flatten()
print(classification_report(np.array(df_test['gender']),preds))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73     43044
           1       0.68      0.67      0.68     36538

    accuracy                           0.71     79582
   macro avg       0.70      0.70      0.70     79582
weighted avg       0.71      0.71      0.71     79582



In [9]:
# load word-embedding based classifier
balanced_gender_clf = pkl.load(open('results/we_bal_g.pkl','rb'))

In [16]:
#load bios data
from sklearn.metrics import classification_report
df_test = pd.read_csv('csvs/multilabel_bios_test.csv')
occs_full = np.unique(df_test['title'])
ind_dict = dict(zip(occs_full, range(len(occs_full))))

In [17]:
# Accuracy of BERT classifier with pre-processing
preds = pr_preds
preds = np.argmax(preds, axis=1).flatten()
print(sklearn.metrics.accuracy_score(np.array([ind_dict[k] for k in df_test['title']]),preds))

0.8440099519991958


In [20]:
# Accuracy of joint BERT classifier

preds = predictions
preds = np.argmax(preds, axis=1).flatten()
print(sklearn.metrics.accuracy_score(np.array([ind_dict[k] for k in df_test['title']]),preds))

0.8547661531502099


In [26]:
# accuracy of decoupled BERT Classifier
f = df_test[df_test['gender']==1]
reset_f = f.reset_index(drop=True)
m = df_test[df_test['gender']==0]
reset_m = m.reset_index(drop=True)
predictions_f= pkl.load(open('results/decoupled_predictions_bert.pkl','rb'))
predictions_f = scipy.special.softmax(predictions_f)
predictions_m= pkl.load(open('results/decoupled_predictions_bert_m.pkl','rb'))
predictions_m = scipy.special.softmax(predictions_m)
print(sklearn.metrics.accuracy_score(np.array([ind_dict[k] for k in f['title']]),np.argmax(predictions_f,axis=1)))
print(sklearn.metrics.accuracy_score(np.array([ind_dict[k] for k in m['title']]),np.argmax(predictions_m,axis=1)))


0.8539328917838962
0.8480624477279063


# get rms gaps

In [27]:
#takes as input a dictionary, outputs accuracy for each subgroup
def pred_summary(pred_dic, Y_test, G_test):
    #wildcard aggregates over different classes/genders
    wildcard = "*"
    genders = sorted(set(G_test)) + [wildcard]
    ys = sorted(set(Y_test)) + [wildcard]
    summary =  {y: {g: [] for g in genders} for y in ys} 
    for y in ys:
        for gender in genders:
            #print(y,sum(array(Y_test)==y))
            #accuracies = []
            bool_subgroup = [y_pred==y_true for y_pred, y_true, g in zip(pred_dic, Y_test, G_test) 
                        if (gender==wildcard or gender==g) and (y==wildcard or y==y_true)]
            acc=np.mean(bool_subgroup)
            summary[y][gender]=acc    
    return summary

In [29]:
#get rmsgap for decoupled
m_summary = pred_summary(np.argmax(predictions_m,axis=1), np.array([ind_dict[k] for k in m['title']]), ['M']*len(m['title']))
f_summary = pred_summary(np.argmax(predictions_f,axis=1), np.array([ind_dict[k] for k in f['title']]), ['F']*len(m['title']))

dec_gaps =[]
for k,v in m_summary.items():
    if k is not '*':
        print(k)
        dec_gaps.append(f_summary[k]['F']-m_summary[k]['M'])
dec_gaps =np.array(dec_gaps)
np.sqrt(np.sum(np.array(dec_gaps)**2)/28)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


0.21844847261386127

In [31]:
#get rmsgap for pre-prpocessing

summary = pred_summary(np.argmax(pr_preds,axis=1), np.array([ind_dict[k] for k in df_test['title']]), df_test['gender'])
pr_gaps =[]
for k,v in summary.items():
    if k is not '*':
        print(k)
        pr_gaps.append(summary[k][0]-summary[k][1])
pr_gaps =np.array(pr_gaps)
np.sqrt(np.sum(np.array(pr_gaps)**2)/28)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


0.10836595096949522

# Cov(r_c, p_c) calculations

In [22]:
# load word embeddings
with open("../../words/embeddings/OtherFormats/crawl-300d-2M.pkl", "rb") as f:
    E = pkl.load(f)
print("Loaded", len(E), "words")


def sim(w1, w2):
    return E[w1].dot(E[w2])/np.linalg.norm(E[w1])/np.linalg.norm(E[w2])
 
sim('he', 'she')


def word_vector_featurize_simple(text, Emb = E):
    return np.mean([Emb[w] for w in re.split(r"[\s\.\!\?\:,\"“\—\-\(\)]+", text) if len(w)>1 and w in Emb], axis=0)



Loaded 1999995 words


In [32]:
f_frac = np.load('results/f_frac.npy')


In [92]:
# for decoupled

G_test = np.array(df_test["gender"])
decoupled_occ = []
for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
    
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]
    gender_probs = balanced_gender_clf.predict_proba(cand_women_X_full)[:,0]
    
    
    Y_test = [ind_dict[p] for p in df_test["title"]]
#     print(np.unique(np.array(Y_test)))
    y_pred_f = predictions_f[:,occ_ind]
    y_pred_m = predictions_m[:,occ_ind]
    
    
    all_preds = np.zeros([len(Y_test)])
    all_preds[list(f.index)] = y_pred_f
    all_preds[list(m.index)] = y_pred_m
    
    occ_probs = all_preds[cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
#     print("%s %s corr: %.3f, p-value %.3f" % (target_occ,' '*(20-len(target_occ)),corr, scipy.stats.kendalltau(occ_probs,gender_probs)[1]))
    decoupled_occ.append(corr)
np.cov(f_frac,decoupled_occ)[0,1]


0.024005407377595645

In [120]:

# for pr
pr_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
        
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

    gender_probs = balanced_gender_clf.predict_proba(cand_women_X_full)[:,0]
    
    occ_probs = pr_preds[:,occ_ind][cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    pr_occ.append(corr)
np.cov(f_frac,pr_occ)[0,1]


0.019997575222467943

In [121]:
# for joint (PO)
joint_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
        
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

    gender_probs = balanced_gender_clf.predict_proba(cand_women_X_full)[:,0]
    
    occ_probs = predictions[:,occ_ind][cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    joint_occ.append(corr)
print(np.cov(f_frac,joint_occ)[0,1])


0.02589993808209482


In [111]:
# for joint (PO)
pr_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
        
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

#     gender_probs = balanced_gender_clf.predict_proba(cand_women_X_full)[:,0]
    
    occ_probs = predictions[:,occ_ind][cand_inds]
    gender_probs = gender_predictions[:,0][cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    pr_occ.append(corr)
print(np.cov(f_frac,pr_occ)[0,1])


0.004837378692504763


In [ ]:
# for joint (PO)
pr_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
        
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

    gender_probs = balanced_gender_clf.predict_proba(cand_women_X_full)[:,0]
    
    occ_probs = predictions[:,occ_ind][cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    pr_occ.append(corr)
np.cov(f_frac,pr_occ)[0,1]


In [61]:

vectorizers=pkl.load(open('results/bow_chi_vectorizers_or.pkl','rb'))

occs = ["surgeon", "software_engineer", "composer", "pastor", "comedian", "architect", "chiropractor", "accountant", "attorney", "filmmaker", "physician", "dentist", "photographer", "professor", "painter", "journalist", "poet", "personal_trainer", "teacher", "psychologist",  "model", "interior_designer", "yoga_teacher", "nurse", "dietitian"]

from scipy.stats import chi2
prob = 0.99
critical = chi2.ppf(prob, 1)
occupation_embeddings = {}

thr = critical
for occ in occs:
    
    all_words = list(vectorizers[occ][thr].vocabulary_.keys())
    print(len(all_words))
    new_dict = {}
    for w in all_words:
        if w in E:
            new_dict[w] = E[w]
    occupation_embeddings[occ+"%.2f"%thr] = new_dict

    
def word_vector_featurize(text, occupation, Emb = E):
    return mean([occupation_embeddings[occupation + "%.2f"%critical][w] for w in re.split(r"[\s\.\!\?\:,\"“\—\-\(\)]+", text) if len(w)>1 and w in occupation_embeddings[occupation+ "%.2f"%critical]], axis=0)


3297
3634
3542
3835
3667
3427
3779
3420
2198
3364
1150
2236
2201
1537
2979
2619
3166
3818
2865
2923
3555
3923
3915
3401
4003


# get corrs w g_irrev

In [62]:
# function to get word embedding of each bio, including only task-irrelevant words
def process_bios(bio_dict,occ):
    irrev_corr_X_train = [word_vector_featurize(p["bio"].lower(), occ, E) for i,p in bio_dict.iterrows()]
    train_inds = []
    for i,p in enumerate(irrev_corr_X_train):
        if isinstance(p, np.ndarray):
            train_inds.append(i)
    irrev_corr_X_train_new = [irrev_corr_X_train[i] for i in range(len(bio_dict)) if i in train_inds]
    return irrev_corr_X_train_new, train_inds

In [63]:
chi_gender_classers = pkl.load(open('results/we_chi_gender_classers_or.pkl','rb'))


In [69]:
chi_gender_classers[target_occ][thr]

SGDClassifier(loss='log')

In [95]:
pre_corr = []
joint_corr = []
decoupled_corr =[]
for i_index, target_occ in enumerate(occs):
    occ_ind = ind_dict[target_occ]
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
    
#     cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]
    cand_women_X,inds = process_bios(cand_women, target_occ) #removes bios that are null when take out words
#     cand_women_X_full = np.array(cand_women_X_full)[inds] #remove them from the full one
    cand_inds = cand_inds[inds]
    gender_probs = chi_gender_classers[target_occ][thr].predict_proba(cand_women_X)[:,0]
    occ_probs = predictions[:,occ_ind][cand_inds]

    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    joint_corr.append(corr)

    occ_probs = pr_preds[:,occ_ind][cand_inds]

    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    pre_corr.append(corr)

    decoupled_inds = np.array(reset_f.loc[reset_f['title']==target_occ].index)[inds]
    occ_probs = predictions_f[:,occ_ind][decoupled_inds]

    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    decoupled_corr.append(corr)
    
print(np.cov(f_frac,pre_corr)[0,1])
print(np.cov(f_frac,joint_corr)[0,1])
print(np.cov(f_frac,decoupled_corr)[0,1])



/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value en

0.002711358125551234
0.006919011441870298
0.0031530784825718806


In [88]:
print(np.cov(f_frac,pre_corr)[0,1])
print(np.cov(f_frac,joint_corr)[0,1])
print(np.cov(f_frac,decoupled_corr)[0,1])


0.0008809819459988751
0.006799586250270358
0.0034634243742574216


In [89]:

print(pearsonr(f_frac,pre_corr)[0])
print(pearsonr(f_frac,joint_corr)[0])
print(pearsonr(f_frac,decoupled_corr)[0])


0.02364793828046382
0.18500395898918956
0.0893235850465698


In [81]:
[x=='F' for x in gender_probs]

[True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 F

In [123]:

# print(pearsonr(f_frac,joint_occ[:25])[0])
print(pearsonr(f_frac,decoupled_occ[:25])[0])
print(pearsonr(f_frac,pr_occ[:25])[0])

0.4812454547306294
0.42963991680833624
0.4009021411263073


In [125]:

print(pearsonr(f_frac,joint_occ[:25])[0])
print(spearmanr(f_frac,decoupled_occ[:25])[0])
# print(spearmanr(f_frac,pr_occ[:25])[0])

0.4812454547306294
0.4815384615384616


In [101]:
from scipy.stats import spearmanr
print(spearmanr(f_frac,joint_corr[:25])[0])
print(spearmanr(f_frac,decoupled_corr[:25])[0])
print(spearmanr(f_frac,pre_corr[:25])[0])

0.06692307692307692
-0.03538461538461538
-0.04538461538461538


In [124]:
from scipy.stats import pearsonr
print(pearsonr(f_frac,joint_corr[:25])[0])
print(pearsonr(f_frac,decoupled_corr[:25])[0])
print(pearsonr(f_frac,pre_corr[:25])[0])

0.1747719529413357
0.07383587167004851
0.06448287087871646


In [118]:
imbalanced_inds  = np.where(abs(f_frac-0.5)>0.25)[0]
def imb_pearsonr(l):
#     print("Pearson's r")
    print(scipy.stats.pearsonr(f_frac[imbalanced_inds],np.array(l)[imbalanced_inds])[0])
    
def imb_cov(l):
#     print("Covariance")
    print(np.cov(f_frac[imbalanced_inds],np.array(l)[imbalanced_inds])[0,1]) 
    
def imb_spear(l):
#     print("Spearman's r")
    print(scipy.stats.spearmanr(f_frac[imbalanced_inds],np.array(l)[imbalanced_inds])[0])

In [119]:

imb_pearsonr(joint_corr)
imb_pearsonr(decoupled_corr)
imb_pearsonr(pre_corr)

imb_spear(joint_corr)
imb_spear(decoupled_corr)
imb_spear(pre_corr)

imb_cov(joint_corr)
imb_cov(decoupled_corr)
imb_cov(pre_corr)

0.2830772656490382
0.16700496180950533
0.164843601635229
0.03636363636363637
-0.1272727272727273
-0.02727272727272728
0.018831260858120563
0.011256762232153213
0.010080136682544873


In [122]:

imb_pearsonr(joint_occ)
imb_pearsonr(decoupled_occ)
imb_pearsonr(pr_occ)

imb_spear(joint_occ)
imb_spear(decoupled_occ)
imb_spear(pr_occ)

imb_cov(joint_occ)
imb_cov(decoupled_occ)
imb_cov(pr_occ)

0.6268657470315026
0.5886323224904346
0.5621829616007565
0.5090909090909091
0.4727272727272727
0.3272727272727273
0.06307462380370318
0.05955358220757176
0.04690800887102422


In [127]:
# rev
print(spearmanr(f_frac,joint_occ[:25]))
print(spearmanr(f_frac,decoupled_occ[:25]))
print(spearmanr(f_frac,pr_occ[:25]))

SpearmanrResult(correlation=0.46307692307692305, pvalue=0.01974462362857622)
SpearmanrResult(correlation=0.4815384615384616, pvalue=0.014800630192683392)
SpearmanrResult(correlation=0.43999999999999995, pvalue=0.02773337204061013)


In [126]:
#irrev
print(spearmanr(f_frac,joint_corr[:25]))
print(spearmanr(f_frac,decoupled_corr[:25]))
print(spearmanr(f_frac,pre_corr[:25]))

SpearmanrResult(correlation=0.06692307692307692, pvalue=0.7506056746243597)
SpearmanrResult(correlation=-0.03538461538461538, pvalue=0.8666482296036683)
SpearmanrResult(correlation=-0.04538461538461538, pvalue=0.8294429316309466)


In [131]:
# for decoupled

G_test = np.array(df_test["gender"])
decoupled_occ = []
for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
    
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]
    gender_probs = gender_predictions[:,0][cand_inds]
    
    
    Y_test = [ind_dict[p] for p in df_test["title"]]
#     print(np.unique(np.array(Y_test)))
    y_pred_f = predictions_f[:,occ_ind]
    y_pred_m = predictions_m[:,occ_ind]
    
    
    all_preds = np.zeros([len(Y_test)])
    all_preds[list(f.index)] = y_pred_f
    all_preds[list(m.index)] = y_pred_m
    
    occ_probs = all_preds[cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
#     print("%s %s corr: %.3f, p-value %.3f" % (target_occ,' '*(20-len(target_occ)),corr, scipy.stats.kendalltau(occ_probs,gender_probs)[1]))
    decoupled_occ.append(corr)



TypeError: tuple indices must be integers or slices, not tuple

In [132]:
spearmanr(f_frac,decoupled_occ)

SpearmanrResult(correlation=0.09, pvalue=0.6687736167185602)

In [133]:
# for joint (PO)
pr_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in occs:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
        
    cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

    gender_probs = gender_predictions[:,0][cand_inds]
    
    occ_probs = predictions[:,occ_ind][cand_inds]
    corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
    pr_occ.append(corr)
spearmanr(f_frac,pr_occ)


SpearmanrResult(correlation=0.10461538461538461, pvalue=0.6187190398978742)

In [134]:
# for joint (PO)
pr_occ=[]

G_test = np.array(df_test["gender"])

for target_occ in ['professor']:
    occ_ind = ind_dict[target_occ]
    
    cand_women = df_test[df_test["title"]==target_occ]
    
    
    cand_women = cand_women[cand_women['gender']==1]
    
    cand_inds = np.array(df_test.loc[(df_test['gender']==1) & (df_test['title']==target_occ)].index)
    print(len(cand_inds))
#     cand_women_X_full =  [word_vector_featurize_simple(p["bio"], E) for i,p in cand_women.iterrows()]

#     gender_probs = gender_predictions[:,0][cand_inds]
    
#     occ_probs = predictions[:,occ_ind][cand_inds]
#     corr = scipy.stats.spearmanr(occ_probs,gender_probs)[0]
#     pr_occ.append(corr)
# spearmanr(f_frac,pr_occ)


10677
